In [1]:
# Import technical libraries
import cvxpy as cp
import numpy as np
import numpy.matlib
from scipy import linalg
import matplotlib.pyplot as plt
import ipywidgets as widgets
import scipy 
import scipy.fftpack
import proximal 
from iplabs import IPLabViewer as viewer
import matplotlib
import proximal
%matplotlib widget
import seaborn
import matplotlib as mpl

# Set Matplotlib style
mpl.rcParams['grid.linewidth'] = 0.3
mpl.rcParams['axes.linewidth'] = 1
plt.rcParams["font.family"] = "Times New Roman"

# Plots

The present notebook is used to generating the different plots and toy examples shown in the report and in the presentation.

### Plot of Element-wise Proximal Operators 

In [2]:
# Define some useful functions
nonneg = lambda x: np.maximum(np.zeros_like(x), x)
l1_prox = lambda x, lamb: np.sign(x)*np.maximum(np.abs(x) - lamb, 0)
l2_prox_param = lambda x, lamb: (1 - np.minimum(lamb, np.ones_like(x)))*x
l3_prox_param = lambda x, lamb: np.sign(x)*(np.sqrt(1 + 12*lamb*np.abs(x) -1))/(6*lamb)

In [3]:
# Plot proximals 
lamb = 0.8
vi = np.arange(-10, 10, 0.1)
plt.figure()
plt.title('Element-Wise Proximal Operators ($\lambda = 0.6$)', fontsize = 16)
plt.grid()
plt.xlabel('$\mathbf{v}$', fontsize = 14)
plt.ylabel('$\mathrm{prox}_{\lambda f}(\mathbf{v})$', fontsize = 14)
# Plot nonnegativity constraints
plt.plot(vi, list(map(nonneg, vi)), label = r'$\mathrm{prox}_{\lambda \delta}$')
# Plot l1 proxi
plt.plot(vi, l1_prox(np.copy(vi), 0.6), label = r'$\mathrm{prox}_{\lambda \ell_1}$')
# Plot l2 proxi
plt.plot(vi, l2_prox_param(np.copy(vi), 0.6), label = r'$\mathrm{prox}_{\lambda \ell_2}$')
# Plot l3 proxi
plt.plot(vi, l3_prox_param(np.copy(vi), 0.6), label = r'$\mathrm{prox}_{\lambda \ell_3}$')
plt.axvline(lamb, color='r', linewidth=0.5, label='\u03bb')
plt.axvline(-lamb, color='r', linewidth=0.5, label='-\u03bb')
plt.legend(prop={'size': 12})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Pan', 'Pan axes with left…

### Moreau Envelope Plot

In [4]:
# Declare original array and values for gamma
x = np.linspace(-5,5,1000) 
gammas = [0.01, 0.1, 0.5, 1, 2]

plt.figure()
# Plot original function
plt.plot(x[x>=0],x[x>=0]**2/2,'k',label=r"$f(x) = \frac{x^2}{2} + \delta_{\mathbb{R}_+}(x)$")
plt.axvline(0, color='k')
# Calculate for different parameters
for gamma in gammas:
    moreau = np.zeros_like(x)
    moreau[x>0] = x[x>0]**2/(2*(gamma+1))
    moreau[x<0] = x[x<0]**2/(2*gamma)    
    plt.plot(x,moreau,label=f"$f_{{{gamma}}}(x)$")

# Plot parameters
plt.legend(prop={'size': 12}); plt.axis((-5,5,0,25)); plt.grid()
plt.xticks(np.arange(-5, 5)); plt.yticks(np.arange(0, 25, 2))
plt.xlabel('$\mathbf{v}$', fontsize = 14); plt.ylabel('$\operatorname{M}_{\lambda f}(\mathbf{v})$', fontsize = 14)
plt.title('Moreau Envelope $\operatorname{M}_{\lambda f}(v)$ of $f(x) = \\frac{x^2}{2} + \delta_{\mathbb{R}^+}(x)$', fontsize = 16)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Pan', 'Pan axes with left…

Text(0.5, 1.0, 'Moreau Envelope $\\operatorname{M}_{\\lambda f}(v)$ of $f(x) = \\frac{x^2}{2} + \\delta_{\\mathbb{R}^+}(x)$')

### Proximal Minimization Example

In [13]:
plt.figure()
# Declare initial arrays, parameters and starting point
x = np.linspace(-5,5,1000) 
gammas = [1, 2, 3]
v = 4
ax = plt.gca()
# Get objective functions with different parameters
for gamma in gammas:
    proximal = np.zeros_like(x)
    proximal = np.abs(x) + (x-v)**2/(2*gamma)
    opt = np.where(proximal == np.min(proximal))[0][0]
    color = next(ax._get_lines.prop_cycler)['color']
    plt.plot(x[opt], proximal[opt], 'o', color = color)
    plt.plot(x,proximal,label=r"$|x| + \frac{1}{2\gamma}\|x-v\|_2^2$" + f', $\lambda$ = {gamma}', color = color)
# Plot
plt.plot(x, np.abs(x), 'k', label=r"$|x|$")
plt.axvline(4, color='r', ls = '--', linewidth=0.5, label='$v$')
plt.legend(prop={'size': 12})
plt.axis((-5,5,0,25))
plt.ylim([0, 10])
plt.xticks(np.arange(-5, 5))
plt.xlabel('$x$', fontsize = 14)
plt.ylabel('$f(x)$', fontsize = 14)
plt.grid()
plt.title(r'Proximal Minimization Algorithm ($f(x) = |x|$, $v = 4$)', fontsize = 16)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Pan', 'Pan axes with left…

Text(0.5, 1.0, 'Proximal Minimization Algorithm ($f(x) = |x|$, $v = 4$)')

## CVXPy Minimization Example

In [12]:
# Define size of vector, lambda, and initialize vector to apply prox to  
n = 10; lambda_ = 0.5
v = np.random.normal(size = (n))
print(f'Value of v: {np.round(v, 3)}')

# Define CVXPy variable of the same size as v, constrained to the nonnegative domain
x_nonneg = cp.Variable(n, nonneg = True)

# Define cost function: |x|_1 + |x-v|^2_2 * (1/2 lambda_)
obj = cp.norm(x_nonneg, 1) + cp.norm(x_nonneg - v, 2)**2/(2*lambda_)

# Solve problem
prox_nonneg_prob = cp.Problem(cp.Minimize(obj))
prox_nonneg_prob.solve()
print(f'The optimal value of x: {np.round(x_nonneg.value, 3)}')

Value of v: [-1.24   1.013  0.106 -1.131 -1.865 -0.641  0.685  0.469 -0.992  0.108]
The optimal value of x: [0.    0.513 0.    0.    0.    0.    0.185 0.    0.    0.   ]


In [13]:
plt.figure(figsize = (8, 3))
plt.plot(v, 'o', label = '$\mathbf{x}$', markersize = 6)
plt.plot(x_nonneg.value, 's', label = '$\mathrm{prox}_{\ell_1 + \delta}(\mathbf{x})$', alpha = 1 ,markersize = 6)
plt.xlabel('Index', fontsize = 14); plt.ylabel('Value', fontsize = 14); plt.title('$\mathrm{prox}_{\ell_1 + \delta}(\mathbf{v})$ Example', fontsize = 16)
plt.vlines([5, 4, 8], [v[5], v[4], v[8]], [x_nonneg.value[5], x_nonneg.value[4], x_nonneg.value[8]], color='r', ls = '--', linewidth=0.8, label = '$\lambda$')
plt.grid()
plt.legend()
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Pan', 'Pan axes with left…